<a href="https://colab.research.google.com/github/shubha045/Project-/blob/main/crewai_product_analyser_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 🛍️ Smart Product Comparison Bot using CrewAI & Gradio

## 📦 Install Required Packages
!pip install crewai openai playwright bs4 serpapi gradio
!pip install crewai-tools
!playwright install

Playwright Host validation warning: 
╔══════════════════════════════════════════════════════╗
║ Host system is missing dependencies to run browsers. ║
║ Missing libraries:                                   ║
║     libwoff2dec.so.1.0.2                             ║
║     libgstgl-1.0.so.0                                ║
║     libgstcodecparsers-1.0.so.0                      ║
║     libavif.so.13                                    ║
║     libharfbuzz-icu.so.0                             ║
║     libenchant-2.so.2                                ║
║     libsecret-1.so.0                                 ║
║     libhyphen.so.0                                   ║
║     libmanette-0.2.so.0                              ║
╚══════════════════════════════════════════════════════╝
    at validateDependenciesLinux (/usr/local/lib/python3.11/dist-packages/playwright/driver/package/lib/server/registry/dependencies.js:269:9)
    at process.processTicksAndRejections (node:internal/process/task_queues:105

In [ ]:
import os
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get ('OPENAI_API_KEY')
os.environ['SERPER_API_KEY'] = userdata.get('SERPER_API_KEY')

In [ ]:
## Setup Tools
from crewai_tools import SerperDevTool, ScrapeWebsiteTool
from crewai import Agent, Task, Crew

search_tool = SerperDevTool()
scraper_tool = ScrapeWebsiteTool()

In [ ]:
## 🤖 Define Agents
user_interaction_agent = Agent(
    role="User Interaction Agent",
    goal="Collect product type and user preferences like price range, brand, and features.",
    backstory="Expert in interacting with users to gather accurate preferences.",
    allow_delegation=False
)

data_extraction_agent = Agent(
    role="Data Extraction Agent",
    goal="Search and extract product information and reviews from the web.",
    backstory="You are skilled in retrieving accurate product data and reviews from websites.",
    tools=[search_tool, scraper_tool],
    allow_delegation=False
)


In [ ]:
comparison_agent = Agent(
    role="Comparison Agent",
    goal="Compare products based on features and user preferences.",
    backstory="Expert in compareing products based on user preferences",
    allow_delegation=False
)

recommendation_agent = Agent(
    role="Recommendation Agent",
    goal="Recommend the best product based on comparisons.",
    backstory="You are expert in recommending products based on comparison.",
    allow_delegation=False
)

In [ ]:
def define_tasks(product_type, price_range, brand_preference, specific_features):
    return [
        Task(
            description=f"Gather product type: {product_type}, price range: {price_range}, brand: {brand_preference}, features: {specific_features}.",
            agent=user_interaction_agent
        ),
        Task(
            description=f"Use SerperDevTool to search for {product_type} reviews and data online.",
            agent=data_extraction_agent
        ),
        Task(
            description=f"Use ScrapeWebsiteTool to extract detailed product info and reviews about {product_type}.",
            agent=data_extraction_agent
        ),
        Task(
            description=f"Compare {product_type} using features like {specific_features} and reviews.",
            agent=comparison_agent
        ),
        Task(
            description=f"Recommend the best {product_type} based on user preferences and comparisons.",
            agent=recommendation_agent
        )
    ]



In [ ]:
def run_crew(product_type, price_range, brand_preference, specific_features):
    tasks = define_tasks(product_type, price_range, brand_preference, specific_features)

    crew = Crew(
        agents=[user_interaction_agent, data_extraction_agent, comparison_agent, recommendation_agent],
        tasks=tasks
    )

    result = crew.kickoff(inputs={
        "product_type": product_type,
        "price_range": price_range,
        "brand_preference": brand_preference,
        "specific_features": specific_features
    })

    return result


In [ ]:
import gradio as gr

def compare_bot_ui(product_type, price_range, brand_preference, specific_features):
    return run_crew(product_type, price_range, brand_preference, specific_features)

gr.Interface(
    fn=compare_bot_ui,
    inputs=[
        gr.Textbox(label="Product Type (e.g., smartphone, headphone)"),
        gr.Textbox(label="Price Range (e.g., 20000-30000 INR)"),
        gr.Textbox(label="Brand Preference (e.g., Samsung, Apple)"),
        gr.Textbox(label="Specific Features (e.g., battery life, camera, noise canceling)")
    ],
    outputs=gr.Textbox(label="Recommended Product"),
    title="🛍️ Smart Product Recommender",
    description="Enter your preferences and get the best product recommendation based on reviews and features!"
).launch()


/usr/local/lib/python3.11/dist-packages/websockets/legacy/__init__.py:6: DeprecationWarning: websockets.legacy is deprecated; see https://websockets.readthedocs.io/en/stable/howto/upgrade.html for upgrade instructions
  warnings.warn(  # deprecated in 14.0 - 2024-11-09
/usr/local/lib/python3.11/dist-packages/uvicorn/protocols/websockets/websockets_impl.py:17: DeprecationWarning: websockets.server.WebSocketServerProtocol is deprecated
  from websockets.server import WebSocketServerProtocol


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://86d6b1d0022ff8d98d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
